In [1]:
import numpy as np
import sympy as sym
import networkx as nx
import matplotlib.pyplot as plt
from dimod import BinaryQuadraticModel, ExactSolver

In [6]:
dim = 5
edges = np.array([[0, 1, 1, 0, 1], [0, 0, 1, 1, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0 ,0, 0, 0]])
w = np.array([[0, 7, 1, 0, 6], [0, 0, 1, 8, 2], [0, 0, 0, 7, 0], [0, 0, 0, 0, 3], [0, 0, 0, 0, 0]])


In [11]:
x = np.zeros((dim, dim), dtype=object)
for i in range(dim):
    for j in range (dim):
        x[i][j] = sym.Symbol(f"x_{i}_{j}", bool=True)

In [12]:
f_1_1 = 0
f_1 = 0
for v in range (dim):
    f_1_1 = np.sum(x[v][:])
    f_1 += sym.expand((1-f_1_1)**2)
    f_1_1 = 0

In [13]:
f_2_1 = 0
f_2 = 0
for j in range (dim):
    f_2_1 = np.sum(x[:][j])
    f_2 += sym.expand((1-f_2_1)**2)

In [14]:
f_3 = 0
for u in range (dim):
    for v in range (u+1, dim):
        for j in range (dim-1):
            f_3 += (1-edges[u][v])*x[u][j]*x[v][j+1]
            

In [15]:
f_4_1 = 0
f_4 = 0
for u in range (dim):
    for v in range (u+1, dim):
        for j in range (dim-1):
            f_4_1 += x[u][j]*x[v][j+1]
        f_4 = w[u][v]*edges[u][v]*f_4_1
        f_4_1 = 0

In [16]:
A=1
B=1
f_t = A*f_1 + A*f_2 + A*f_3 + B*f_4

In [17]:
f_t

2*x_0_0**2 + 4*x_0_0*x_0_1 + 4*x_0_0*x_0_2 + 4*x_0_0*x_0_3 + 4*x_0_0*x_0_4 - 4*x_0_0 + 2*x_0_1**2 + 4*x_0_1*x_0_2 + 4*x_0_1*x_0_3 + 4*x_0_1*x_0_4 - 4*x_0_1 + 2*x_0_2**2 + 4*x_0_2*x_0_3 + 4*x_0_2*x_0_4 - 4*x_0_2 + 2*x_0_3**2 + 4*x_0_3*x_0_4 - 4*x_0_3 + 2*x_0_4**2 - 4*x_0_4 + 2*x_1_0**2 + 4*x_1_0*x_1_1 + 4*x_1_0*x_1_2 + 4*x_1_0*x_1_3 + 4*x_1_0*x_1_4 - 4*x_1_0 + 2*x_1_1**2 + 4*x_1_1*x_1_2 + 4*x_1_1*x_1_3 + 4*x_1_1*x_1_4 - 4*x_1_1 + 2*x_1_2**2 + 4*x_1_2*x_1_3 + 4*x_1_2*x_1_4 - 4*x_1_2 + 2*x_1_3**2 + 4*x_1_3*x_1_4 - 4*x_1_3 + 2*x_1_4**2 - 4*x_1_4 + 2*x_2_0**2 + 4*x_2_0*x_2_1 + 4*x_2_0*x_2_2 + 4*x_2_0*x_2_3 + 4*x_2_0*x_2_4 - 4*x_2_0 + 2*x_2_1**2 + 4*x_2_1*x_2_2 + 4*x_2_1*x_2_3 + 4*x_2_1*x_2_4 - 4*x_2_1 + 2*x_2_2**2 + 4*x_2_2*x_2_3 + 4*x_2_2*x_2_4 - 4*x_2_2 + 2*x_2_3**2 + 4*x_2_3*x_2_4 - 4*x_2_3 + 2*x_2_4**2 - 4*x_2_4 + 2*x_3_0**2 + 4*x_3_0*x_3_1 + 4*x_3_0*x_3_2 + 4*x_3_0*x_3_3 + 4*x_3_0*x_3_4 + 3*x_3_0*x_4_1 - 4*x_3_0 + 2*x_3_1**2 + 4*x_3_1*x_3_2 + 4*x_3_1*x_3_3 + 4*x_3_1*x_3_4 + 3*x_3_1*x_4

In [18]:
Q = np.zeros((dim, dim), dtype=float)
for i in range (dim):
    for j in range(i, dim):
        Q[i][j] = float(f_t.coeff(x[i][j]*x[i][j]))

In [19]:
bqm       = BinaryQuadraticModel.from_qubo(Q)
sampler   = ExactSolver()
sampleset = sampler.sample(bqm)
result    = sampleset.first